In [ ]:
# from urllib.request import urlopen

# r = urlopen('https://case.edu/projects/erpextract/soc.xml')
# mytree = ET.parse(r)

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

xtree = ET.parse('soc.xml')
xroot = xtree.getroot()
print(xroot)

In [ ]:
croot = xroot.find('Terms/Term/Classes')

cchild = list(croot)

cols = []
for child in cchild[0]:
#     print(child.tag)
    cols.append(child.tag)

In [ ]:
target_cols = ['Subject', 'CatalogNbr', 'CourseTitleLong', 'Components/Component/Component', 'Dates', 'UnitsRange',
               'Meetings/Meeting/DaysTimes', 'Meetings/Meeting/Room', 'Meetings/Meeting/Instructor',
              'EnrollmentCapacity', 'EnrollmentTotal', 'AvailableSeats']

In [ ]:
course_list = []
for child in cchild:
    course_detail = []
    for col in target_cols:
        if child.find(col) != None:
            course_detail.append(child.find(col).text)
        else:
            course_detail.append(np.nan)
    course_list.append(course_detail)

In [ ]:
cols = ['Subject', 'CatalogNumber', 'CourseTitle', 'CourseType','Dates', 'Credit', 'MeetingTime', 'Room', 'Instructor', 'TotalSeats', 'SeatsTaken', 'AvailableSeats']
df = pd.DataFrame(data=course_list, columns=cols)

In [ ]:
df.loc[ ((df['Subject'] == 'CSDS') & ( df['Instructor'] == 'Shuai Xu') ) ]

In [ ]:
cchild[0].find('Components/Component/Component').text

In [ ]:
df.to_csv('courses.csv', index=False)

In [ ]:
df.shape
df.sort_values(by=['CourseType'])

In [ ]:
#first five rows of the dataframe
df.head()

In [ ]:
#provide the information of each column of the dataframe
df.info()

In [ ]:
#describe the statistics of each column of the dataframe
df.describe()

In [ ]:
df.loc[((df['MeetingTime']=='MWF 11:40 AM - 12:30 PM') & (df['Subject']=='CSDS'))]

In [ ]:
#functions necessary for the Scheduling Algorithm

current_schedule = pd.DataFrame()
current_schedule = pd.DataFrame( columns=cols)

current_schedule.loc[df.index[1505]] = df.iloc[1505]
current_schedule.loc[df.index[1523]] = df.iloc[1523]
current_schedule.loc[df.index[1479]] = df.iloc[1479]

display(current_schedule)

In [ ]:
#has_conflict(): returns whether or not there are scheudle conflicts
import re

def has_conflict(schedule):
    is_conflict = false;
    n = schedule.shape[0];
    r1=1;r2=r1+1
    for _ in range(n*(n+1)/2):
        cs1 = schedule.iloc[r1,6]; cs2 = schedule.iloc[r2,6]
        #case 1: meeting time is exactly the same
        if cs1 == cs2:
            is_conflict = true;
            break
        #case 2: meeting time is not exactly the same but time conflict occurs
        elif cs1[0:4] == cs2[0:4]:
            digits1 = re.findall(r'\d+', cs1); digits2 = re.findall(r'\d+', cs2);
            if 
            
        
            

In [ ]:
#is_available() returns yes or no if the class is available to register
def is_available(schedule, new_class):
    available = true;
    for _ in range(schedule.shape[0]):
        if new_class.iloc[1,-1] <= 0:
            available = false
            break
    return available

In [ ]:
#time_difference() returns the end time of one class - starting time of another in the same day
def time_difference(class1, class2):
    meeting_time1 = class1.iloc[1,6]; meeting_time2 = class2.iloc[1,6];
    digits1 = re.findall(r'\d+', meeting_time1); digits2 = re.findall(r'\d+', meeting_time2);
    